In [1]:
from model import *
import pytchat


C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

In [4]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

100%|██████████| 4102/4102 [00:04<00:00, 1022.18it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2444.48it/s]
4102it [00:01, 2357.64it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1253.28it/s]


...

Validating length of dkp_frames
Validating length of kp_frames
Adding deltas to frames...


4102it [00:02, 1834.32it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 36618.53it/s]


Validating length of data after emotions
frame_dim: 107


In [5]:
def get_video_by_emotion(data, specific_emotion):
    """
    Retrieve a single video of a specified emotion from the dataset.
    
    Parameters:
    - data (list): The entire dataset, assumed to be a list of videos.
    - specific_emotion (str): The emotion label of the desired video.
    
    Returns:
    - list: A single video corresponding to the specified emotion.
    """
    # Convert the specific emotion to its encoding
    specific_emotion_tuple = tuple(emotion_to_encoding(specific_emotion))
    
    # Search the dataset for a video with the specified emotion - return as tensor
    for video in data:
        if tuple(video[0][-7:]) == specific_emotion_tuple:
            return torch.tensor([video]).to(device).float()
    
    # If no video with the specified emotion is found, return None
    return None




# Example usage
specific_emotion_video = get_video_by_emotion(train_data, "Anger")

In [6]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [7]:
chat = pytchat.create(video_id="jfKfPfyJRdk")

# Initialize a dictionary to keep track of cumulative scores and counts for each emotion
emotion_data = {emotion: {"score": 0.0, "count": 0} for emotion in emotion_labels}

# List to store average scores indexed by emotions
average_scores = [0.0 for _ in emotion_labels]
length =0


# create model
CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"

m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)

# initial generation always neutral
generated = m.generate(specific_emotion_video, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)

# visualize
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)




Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth


Visualizing frames: 100%|██████████| 300/300 [00:33<00:00,  9.02it/s]


In [ ]:
import threading

In [9]:


while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        result = pipe(c.message)  # Assuming pipe() returns emotion prediction
        print(result)
        
        # Update emotion data with new score and count
        emotion = result[0]['label']
        score = result[0]['score']
        emotion_data[emotion]["score"] += score
        emotion_data[emotion]["count"] += 1
        
        # Compute average scores for each emotion
        for i, emotion in enumerate(emotion_labels):
            if emotion_data[emotion]["count"] > 0:
                average_scores[i] = emotion_data[emotion]["score"] / emotion_data[emotion]["count"]
        
        # Print average scores (can also store them elsewhere if needed)
        print("Average scores:", average_scores)
        length  +=1
        
        if length % 10 ==0 :
            # prep new input from chat emotion average scores and last 16 frames of generated
            last_frames = unnorm_out[0][-16:]
            new_batch = [frame[:-7] + average_scores for frame in last_frames]
            new_batch = torch.tensor([new_batch]).to(device).float()

            # generate new frames
            generated = m.generate(new_batch, FRAMES_GENERATE)
            unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)

            # visualize
            visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)


2023-10-19 19:07:34 [Petra Miller]- U ONLY PROPEL MY TRUTH
[{'label': 'anger', 'score': 0.674485981464386}]
Average scores: [0.674485981464386, 0.0, 0.0, 0.6622929573059082, 0.8021373748779297, 0.0, 0.0]
2023-10-19 19:07:36 [Ted.]- Rahul :grinning_cat_with_smiling_eyes:
[{'label': 'joy', 'score': 0.49819064140319824}]
Average scores: [0.674485981464386, 0.0, 0.0, 0.6075921853383383, 0.8021373748779297, 0.0, 0.0]
2023-10-19 19:07:37 [Vivaldi]- lofi is a dreams crusher 
[{'label': 'neutral', 'score': 0.9646530151367188}]
Average scores: [0.674485981464386, 0.0, 0.0, 0.6075921853383383, 0.8201946682400174, 0.0, 0.0]
2023-10-19 19:07:40 [doneswar]- :cat_with_tears_of_joy:
[{'label': 'joy', 'score': 0.6010401248931885}]
Average scores: [0.674485981464386, 0.0, 0.0, 0.6059541702270508, 0.8201946682400174, 0.0, 0.0]
2023-10-19 19:07:40 […RO'A]- :slightly_smiling_face::slightly_smiling_face:
[{'label': 'neutral', 'score': 0.6915953755378723}]
Average scores: [0.674485981464386, 0.0, 0.0, 0.605

Visualizing frames: 100%|██████████| 300/300 [00:33<00:00,  9.00it/s]


2023-10-19 19:07:51 [Flower 🌸 ]- :slightly_smiling_face:
[{'label': 'neutral', 'score': 0.8979910016059875}]
Average scores: [0.5986655056476593, 0.0, 0.0, 0.6253223896026612, 0.8243570498057774, 0.0, 0.0]
2023-10-19 19:07:51 [Sereñe]- hmm
[{'label': 'neutral', 'score': 0.9865169525146484}]
Average scores: [0.5986655056476593, 0.0, 0.0, 0.6253223896026612, 0.8351677099863688, 0.0, 0.0]
2023-10-19 19:07:52 [Hinna]- Tuff Danny said Tuff gone now I'm crying in loud speakers :skull::skull::skull::skull::skull::skull::skull::skull:
[{'label': 'sadness', 'score': 0.7676039934158325}]
Average scores: [0.5986655056476593, 0.0, 0.0, 0.6253223896026612, 0.8351677099863688, 0.7676039934158325, 0.0]
2023-10-19 19:07:53 [Petra Miller]- OF COURSE FLOWER
[{'label': 'neutral', 'score': 0.9794634580612183}]
Average scores: [0.5986655056476593, 0.0, 0.0, 0.6253223896026612, 0.8441861942410469, 0.7676039934158325, 0.0]
2023-10-19 19:07:55 [s...☆.]- عدووووو ما اجااا.؟ :moai:
[{'label': 'neutral', 'score':

Visualizing frames: 100%|██████████| 300/300 [00:34<00:00,  8.63it/s]


2023-10-19 19:08:02 [𝑺𝑨𝑴𝑬𝑹]- وعليكم السلام
[{'label': 'neutral', 'score': 0.8485379219055176}]
Average scores: [0.5986655056476593, 0.0, 0.0, 0.6537298659483591, 0.8703334846279838, 0.7676039934158325, 0.0]


KeyboardInterrupt: 

In [7]:

while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        result = pipe(c.message)  # Assuming pipe() returns emotion prediction
        print(result)
        
        # Update emotion data with new score and count
        emotion = result[0]['label']
        score = result[0]['score']
        emotion_data[emotion]["score"] += score
        emotion_data[emotion]["count"] += 1
        
        # Compute average scores for each emotion
        for i, emotion in enumerate(emotion_labels):
            if emotion_data[emotion]["count"] > 0:
                average_scores[i] = emotion_data[emotion]["score"] / emotion_data[emotion]["count"]
        
        # Print average scores (can also store them elsewhere if needed)
        print("Average scores:", average_scores)
        length  +=1
        
        if length % 10 ==0 :
            generated = m.generate(xb, FRAMES_GENERATE)


2023-10-19 18:55:18 [ Ar. Sharma~]- :cigarette:
[{'label': 'neutral', 'score': 0.9502205848693848}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.9502205848693848, 0.0, 0.0]
2023-10-19 18:55:18 [Michael Terry Wilson]- Best wishes and love to everyone.
[{'label': 'neutral', 'score': 0.682776689529419}]
Average scores: [0.0, 0.0, 0.0, 0.0, 0.8164986371994019, 0.0, 0.0]


KeyboardInterrupt: 

In [ ]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

In [2]:


CHECKPOINT_PATH = "checkpoints/proto5_checkpoint.pth"

# create model
m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)
# Generate a sequence
print(f'Generating sequence of {FRAMES_GENERATE} frames...')

generated = m.generate(, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)



100%|██████████| 4102/4102 [00:03<00:00, 1086.69it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2499.12it/s]
4102it [00:01, 2397.84it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1039.36it/s]


...

4102it [00:03, 1110.06it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 9863.41it/s]


Validating length of data after emotions
frame_dim: 107
Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth
Generating sequence of 300 frames...


NameError: name 'batch' is not defined

In [19]:
def encoding_to_emotions(encoding):
    """
    Convert an encoding to its corresponding emotion labels.
    
    Parameters:
    - encoding (list of int): The encoding of the emotions.
    
    Returns:
    - list of str: The labels of the emotions.
    """
    
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    
    # Find the indices of non-zero values in the encoding and return the corresponding labels
    emotion_indices = [i for i, value in enumerate(encoding) if value > 0]
    return [emotion_labels[i] for i in emotion_indices]

# Test the function with an example encoding
test_encoding = [0.97, 0, 1, 1, 0, 0, 0]
encoding_to_emotions(test_encoding)


['Anger', 'Fear', 'Happiness']

In [12]:
frame[-7:]

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]

In [33]:
# switch emotions of frames 
def switch_emotion(frame,old_emotion,new_emotion):
    """
    Switch the emotion of a frame.
    
    Parameters:
    - frame (list of int): The frame to switch the emotion of.
    - old_emotion (str): The emotion to switch from.
    - new_emotion (str): The emotion to switch to.
    
    Returns:
    - list of int: The frame with the emotion switched.
    """
    
    # Get the indices of the old and new emotion
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    old_emotion_index = emotion_labels.index(old_emotion)
    new_emotion_index = emotion_labels.index(new_emotion)
    
    # Switch the emotion
    frame[-7 + old_emotion_index] = 0
    frame[-7 + new_emotion_index] = 1
    
    return frame

new_batch = []
for frame in batch:
    emotion = encoding_to_emotions(frame[-7:])
    print(emotion)
    new_batch.append(frame[:-7] + emotion_to_encoding('Disgust'))

['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Anger', 'Disgust', 'Happiness', 'Neutral']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Happiness', 'Neutral', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sad', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Fear', 'Happiness']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sur

In [24]:
for batch in unnorm_out:
    visualise_skeleton(batch, max_x, max_y, max_frames=FRAMES_GENERATE,save =True,save_path=None,prefix=f'adam_{EPOCHS}_coord_normalemotion_happy',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:33<00:00,  9.07it/s]


In [25]:
visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytosad',train_seed=train_seed,delta=False)

Visualizing frames:  27%|██▋       | 82/300 [00:09<00:24,  9.06it/s]


KeyboardInterrupt: 

In [34]:
happy_to_sad_xb = [new_batch[-16:]]
happy_to_sad_xb = torch.tensor(happy_to_sad_xb).to(device).float()


generated = m.generate(happy_to_sad_xb, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)


In [35]:
for new_batch in unnorm_out:

    visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.17it/s]


In [36]:
xb.shape

torch.Size([1, 16, 107])